# FLIPKART RECOMMENDING ITEMS

In this notebook, I take a dataset of flipkart products and use it to recommend products based on various features and a similarity score of the same. The dataset has been taken up from kaggle and can be  found here - https://www.kaggle.com/PromptCloudHQ/flipkart-products

In [1]:
import pandas as pd

In [2]:
flipkart = pd.read_csv('flipkartdata.csv')

In [3]:
flipkart.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


A closer look at product category tree !

In [4]:
flipkart.iloc[0,4].strip('[]').strip('"').split(">>")

['Clothing ',
 " Women's Clothing ",
 ' Lingerie, Sleep & Swimwear ',
 ' Shorts ',
 ' Alisha Shorts ',
 " Alisha Solid Women's Cycling Shorts"]

Now my approach is pretty simple and clear. I want to work with three columns to generate recommendations !
1. Product Category Tree
2. Desciption
3. Product Specifiction

Once I have done this, I am going to convert all these three columns into list of words each contains beforehand. What I mean by that is : Let's say the product specification is : {processor : i7, ram : 8GB} I would want to create a list by cleaning the column and list would look something like this : [processor i7, ram 8gb,..]

After doing that I am going to create a vector(of binary values) of a sorted list containing all the words in all three columns respectively. Then I am going to compare all the three binary lists generated with a similarity function (cosine similarity) of a selected item with all the items there are ! Then I am going to display the top items present in on the basis of score. Let's get started !

## 1.1 Cleaning the data

### Product Category Tree

In [5]:
for i,j in zip(flipkart.index,flipkart['product_category_tree']):
    li = j.strip('[]').strip('"').split(">>")
    lis = []
    for k in li:
        k = k.strip(' ')
        lis.append(k)
    flipkart.loc[i,'product_category_tree'] = str(lis)

In [6]:
flipkart.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"['Clothing', ""Women's Clothing"", 'Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"['Furniture', 'Living Room Furniture', 'Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"['Footwear', ""Women's Footwear"", 'Ballerinas',...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"['Clothing', ""Women's Clothing"", 'Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"['Pet Supplies', 'Grooming', 'Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


### Description of the product

In [7]:
flipkart.iloc[3,10]

"Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Black, Red,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 2 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTGHT_11 In the Box 2 shorts"

In [8]:
from nltk.corpus import stopwords

In [9]:
import string

In [10]:
pun = list(string.punctuation)

In [11]:
pun

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [12]:
stop = stopwords.words('english')

In [13]:
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [14]:
remove = pun + stop

In [15]:
# string 'key features' is very common to all the descriptions of the products so I'll be removing them as well
remove = remove + ['key','features']

In [16]:
len(remove)

213

In [17]:
from nltk import word_tokenize

In [18]:
flipkart.dropna(inplace=True)

Below I am just performing some cleanup task of removing stop words, punctuations and checking if a word is numeric or not.

In [19]:
def cleandescription(text):
    words = word_tokenize(text.lower())
    cleaned = [w for w in words if w not in remove and w.isnumeric()==False]
    return " ".join(cleaned)

In [20]:
flipkart['cleaned_description'] = flipkart['description'].apply(cleandescription)

In [21]:
flipkart.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,cleaned_description
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"['Clothing', ""Women's Clothing"", 'Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",alisha solid women 's cycling shorts cotton ly...
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"['Furniture', 'Living Room Furniture', 'Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",fabhomedecor fabric double sofa bed finish col...
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"['Footwear', ""Women's Footwear"", 'Ballerinas',...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",aw bellies sandals wedges heel casuals aw bell...
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"['Clothing', ""Women's Clothing"", 'Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",alisha solid women 's cycling shorts cotton ly...
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"['Pet Supplies', 'Grooming', 'Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",specifications sicons purpose arnica dog shamp...


Now obviously descriptions are long, and I can't personally use every word to find similarity between the products based on the description column. So what I'll be doing now is to get the most important and relevant words present in the description of the product that should theoretically represent the product. To do that I'll be using TFIDF vectorizer by sklearn. You can learn more about it on the internet but as a brief, it is a score assigned to a word found by seeing how much the word appears in a specific document versus how much the word is present in all the documents. So intuitively, when you have a word frequent in a certain document like 'hoodie' and it is not present in other documents, 'hoodie' will receive a good score, but if you have a word like 'good' which is high in frequency in a certain document, it may also be high in frequency in other documents hence you will receive a bad score. You can then choose the top list of words you want to keep, something I am about to do now !

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
v = TfidfVectorizer(max_features=100)

In [24]:
x = v.fit_transform(flipkart['cleaned_description'])

In [25]:
v.get_feature_names()

['additional',
 'alloy',
 'back',
 'best',
 'black',
 'blue',
 'box',
 'bra',
 'brand',
 'buy',
 'car',
 'care',
 'cash',
 'casual',
 'cm',
 'code',
 'collection',
 'color',
 'com',
 'contents',
 'cotton',
 'cover',
 'covers',
 'day',
 'delivery',
 'design',
 'designed',
 'details',
 'diamond',
 'dimensions',
 'easy',
 'fabric',
 'fashion',
 'fit',
 'flipkart',
 'free',
 'full',
 'general',
 'genuine',
 'gold',
 'guarantee',
 'height',
 'high',
 'ideal',
 'inch',
 'india',
 'ipad',
 'jewellery',
 'light',
 'look',
 'made',
 'mat',
 'material',
 'men',
 'metal',
 'mm',
 'model',
 'multicolor',
 'na',
 'neck',
 'necklace',
 'number',
 'occasion',
 'one',
 'online',
 'pack',
 'package',
 'pattern',
 'price',
 'prices',
 'printed',
 'product',
 'products',
 'quality',
 'red',
 'replacement',
 'ring',
 'rs',
 'sales',
 'set',
 'shipping',
 'shirt',
 'silver',
 'size',
 'sleeve',
 'solid',
 'specifications',
 'style',
 'top',
 'type',
 'usb',
 'use',
 'warranty',
 'wash',
 'wear',
 'weight',

In [26]:
print(x[1])

  (0, 86)	0.024319649827120014
  (0, 27)	0.033530778727808304
  (0, 61)	0.029163582171134298
  (0, 31)	0.23836594714001447
  (0, 89)	0.22739599278675268
  (0, 37)	0.04937994377252703
  (0, 11)	0.07777599368788857
  (0, 87)	0.035460748213652124
  (0, 6)	0.027622426083332385
  (0, 17)	0.28682158713657757
  (0, 4)	0.20207494176174307
  (0, 68)	0.02353692830148327
  (0, 77)	0.013786088429122106
  (0, 30)	0.04659009104722718
  (0, 71)	0.6602435123839343
  (0, 35)	0.044228351015709995
  (0, 24)	0.04563594954946608
  (0, 8)	0.03203103373677319
  (0, 52)	0.19400590222480169
  (0, 58)	0.05254094306769071
  (0, 83)	0.03950698569845604
  (0, 56)	0.03555502630711717
  (0, 91)	0.08822983320724906
  (0, 21)	0.12237851085359817
  (0, 63)	0.04600867204842237
  (0, 92)	0.38747861048769944
  (0, 22)	0.05386794505590553
  (0, 29)	0.11006039687726658
  (0, 95)	0.036728555282884885
  (0, 41)	0.0399019250971406
  (0, 55)	0.1966938069211709
  (0, 97)	0.04165034114695169
  (0, 73)	0.07473197225479211
  (0, 33

### Product Specifications

In [27]:
(flipkart.loc[0,'product_specifications'])

'{"product_specification"=>[{"key"=>"Number of Contents in Sales Package", "value"=>"Pack of 3"}, {"key"=>"Fabric", "value"=>"Cotton Lycra"}, {"key"=>"Type", "value"=>"Cycling Shorts"}, {"key"=>"Pattern", "value"=>"Solid"}, {"key"=>"Ideal For", "value"=>"Women\'s"}, {"value"=>"Gentle Machine Wash in Lukewarm Water, Do Not Bleach"}, {"key"=>"Style Code", "value"=>"ALTHT_3P_21"}, {"value"=>"3 shorts"}]}'

This was the hardest one to clean since there isn't much of a pattern to extract information and also the format is that of a string !

In [28]:
x = flipkart.loc[10,'product_specifications'].strip('{"product_specification"=>[]]')

In [29]:
x

'key"=>"Occasion", "value"=>"Casual"}, {"key"=>"Ideal For", "value"=>"Women"}, {"key"=>"Heel Height", "value"=>"0 inch"}, {"key"=>"Outer Material", "value"=>"PU"}, {"key"=>"Color", "value"=>"Black"}]}'

In [30]:
x = '{' + x + '}'

In [31]:
y = x.split('}, ')
y

['{key"=>"Occasion", "value"=>"Casual"',
 '{"key"=>"Ideal For", "value"=>"Women"',
 '{"key"=>"Heel Height", "value"=>"0 inch"',
 '{"key"=>"Outer Material", "value"=>"PU"',
 '{"key"=>"Color", "value"=>"Black"}]}}']

In [32]:
for i in y:
    print(i.strip('{}').replace('key','').replace('"','').replace('=>','').replace('value','').replace(',','').replace('{','').replace('}','').replace('[','').replace(']',''))

Occasion Casual
Ideal For Women
Heel Height 0 inch
Outer Material PU
Color Black


In [33]:
def clean_specification(st):
    x = st.strip('{"product_specification"=>[]]')
    x = '{' + x + '}'
    y = x.split('}, ')
    li = []
    for i in y:
        k = i.strip('{}').replace('key','').replace('"','').replace('=>','').replace('value','').replace(',','').replace('{','').replace('}','').replace('[','').replace(']','')
        if(len(k)<=40):
            li.append(k.lower())
    return li

In [34]:
flipkart['specs']=flipkart['product_specifications'].apply(clean_specification)

In [35]:
flipkart.loc[20,'specs']

['pet type dog',
 'brand sicons',
 'quantity 200 ml',
 'model number sh.df-02',
 'type conditioning',
 'fragrance conditoner',
 'form factor gel']

Further cleaning product_category_tree

In [36]:
flipkart.loc[20,'product_category_tree']

"['Pet Supplies', 'Grooming', 'Skin & Coat Care', 'Shampoo', 'Sicons Conditioning Conditoner Dog Shampoo (200 ml)']"

In [37]:
flipkart['product_category_tree'] = flipkart['product_category_tree'].str.strip('[]')

In [38]:
flipkart.loc[20,'product_category_tree']

"'Pet Supplies', 'Grooming', 'Skin & Coat Care', 'Shampoo', 'Sicons Conditioning Conditoner Dog Shampoo (200 ml)'"

In [39]:
flipkart['product_category_tree'] = flipkart['product_category_tree'].str.replace(' ','')

In [40]:
flipkart.loc[20,'product_category_tree']

"'PetSupplies','Grooming','Skin&CoatCare','Shampoo','SiconsConditioningConditonerDogShampoo(200ml)'"

In [41]:
flipkart['product_category_tree'] = flipkart['product_category_tree'].str.replace("'",'')

In [42]:
flipkart.loc[20,'product_category_tree']

'PetSupplies,Grooming,Skin&CoatCare,Shampoo,SiconsConditioningConditonerDogShampoo(200ml)'

In [43]:
flipkart['product_category_tree'] = flipkart['product_category_tree'].str.replace('"','')

In [44]:
flipkart['product_category_tree'] = flipkart['product_category_tree'].str.split(',')

In [45]:
flipkart.loc[10,'product_category_tree']

['Footwear', 'WomensFootwear', 'Ballerinas', 'LadelaBellies']

In [46]:
for i,j in zip(flipkart.index,flipkart['product_category_tree']):
    lis = []
    lis = j
    lis.sort()
    flipkart.loc[i,'product_category_tree']=str(lis)
flipkart['product_category_tree'] = flipkart['product_category_tree'].str.strip('[]')
flipkart['product_category_tree'] = flipkart['product_category_tree'].str.replace(' ','')
flipkart['product_category_tree'] = flipkart['product_category_tree'].str.replace("'",'')
flipkart['product_category_tree'] = flipkart['product_category_tree'].str.replace('"','')
flipkart['product_category_tree'] = flipkart['product_category_tree'].str.split(',')
flipkart.loc[10,'product_category_tree']

['Ballerinas', 'Footwear', 'LadelaBellies', 'WomensFootwear']

In [47]:
categorylist = []
for i,j in flipkart.iterrows():
    k = j['product_category_tree']
    for x in k:
        if x not in categorylist:
            categorylist.append(x)
categorylist[:10]

['AlishaShorts',
 'AlishaSolidWomensCyclingShorts',
 'Clothing',
 'Lingerie',
 'Shorts',
 'Sleep&Swimwear',
 'WomensClothing',
 'FabHomeDecorFabricDoubleSofaBed(FinishColo...',
 'Furniture',
 'LivingRoomFurniture']

In [48]:
def category_binary(inp):
    li = []
    for i in categorylist:
        if i in inp:
            li.append(1)
        else:
            li.append(0)
    return li
flipkart['product_binary']=flipkart['product_category_tree'].apply(category_binary)

In [49]:
flipkart.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,cleaned_description,specs,product_binary
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[AlishaShorts, AlishaSolidWomensCyclingShorts,...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",alisha solid women 's cycling shorts cotton ly...,"[fabric cotton lycra, type cycling shorts, pat...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,[FabHomeDecorFabricDoubleSofaBed(FinishColo......,SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",fabhomedecor fabric double sofa bed finish col...,"[1 sofa bed, brand fabhomedecor, mattress incl...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, ..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[AWBellies, Ballerinas, Footwear, WomensFootwear]",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",aw bellies sandals wedges heel casuals aw bell...,"[ideal for women, occasion casual, color red, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[AlishaShorts, AlishaSolidWomensCyclingShorts,...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",alisha solid women 's cycling shorts cotton ly...,"[fabric cotton lycra, type cycling shorts, pat...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[Grooming, PetSupplies, Shampoo, SiconsAllPurp...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",specifications sicons purpose arnica dog shamp...,"[pet type dog, brand sicons, quantity 500 ml, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [50]:
for i,j in zip(flipkart.index,flipkart['specs']):
    li = []
    li = j
    li.sort()
    flipkart.loc[i,'specs']=str(li)
    
flipkart['specs'] = flipkart['specs'].str.strip('[]')
flipkart['specs'] = flipkart['specs'].str.replace(' ','')
flipkart['specs'] = flipkart['specs'].str.replace("'",'')
flipkart['specs'] = flipkart['specs'].str.replace('"','')
flipkart['specs'] = flipkart['specs'].str.split(',')    

specstotal = []
for i,j in flipkart.iterrows():
    k = j['specs']
    for x in k:
        if x not in specstotal:
            specstotal.append(x)
specstotal[:10]

['3shorts',
 'fabriccottonlycra',
 'idealforwomens',
 'patternsolid',
 'stylecodealtht_3p_21',
 'typecyclingshorts',
 '1sofabed',
 'bedsizedouble',
 'brandfabhomedecor',
 'deliveryconditionknockdown']

In [51]:
def specs_binary(inp):
    li = []
    for i in specstotal:
        if i in inp:
            li.append(1)
        else:
            li.append(0)
    return li
flipkart['specs_binary']=flipkart['specs'].apply(specs_binary)

In [52]:
flipkart.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,cleaned_description,specs,product_binary,specs_binary
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[AlishaShorts, AlishaSolidWomensCyclingShorts,...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",alisha solid women 's cycling shorts cotton ly...,"[3shorts, fabriccottonlycra, idealforwomens, p...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,[FabHomeDecorFabricDoubleSofaBed(FinishColo......,SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",fabhomedecor fabric double sofa bed finish col...,"[1sofabed, bedsizedouble, brandfabhomedecor, d...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[AWBellies, Ballerinas, Footwear, WomensFootwear]",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",aw bellies sandals wedges heel casuals aw bell...,"[colorred, heelheight1inch, idealforwomen, occ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[AlishaShorts, AlishaSolidWomensCyclingShorts,...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",alisha solid women 's cycling shorts cotton ly...,"[2shorts, fabriccottonlycra, idealforwomens, p...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[Grooming, PetSupplies, Shampoo, SiconsAllPurp...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",specifications sicons purpose arnica dog shamp...,"[brandsicons, formfactorliquid, fragrancearnic...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [53]:
flipkart.loc[0,'cleaned_description']

"alisha solid women 's cycling shorts cotton lycra navy red navy specifications alisha solid women 's cycling shorts shorts details number contents sales package pack fabric cotton lycra type cycling shorts general details pattern solid ideal women 's fabric care gentle machine wash lukewarm water bleach additional details style code altht_3p_21 box shorts"

In [54]:
for i,j in zip(flipkart.index,flipkart['cleaned_description']):
    words = word_tokenize(j)
    words.sort()
    flipkart.loc[i,'cleaned_description']=str(words)

In [55]:
flipkart['cleaned_description'] = flipkart['cleaned_description'].str.strip('[]')
flipkart['cleaned_description'] = flipkart['cleaned_description'].str.replace(' ','')
flipkart['cleaned_description'] = flipkart['cleaned_description'].str.replace("'",'')
flipkart['cleaned_description'] = flipkart['cleaned_description'].str.replace('"','')
flipkart['cleaned_description'] = flipkart['cleaned_description'].str.split(',')    

In [56]:
descriptiontotal = v.get_feature_names()
def desc_binary(inp):
    li = []
    for i in descriptiontotal:
        if i in inp:
            li.append(1)
        else:
            li.append(0)
    return li
flipkart['description_binary']=flipkart['cleaned_description'].apply(desc_binary)

In [57]:
flipkart.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,cleaned_description,specs,product_binary,specs_binary,description_binary
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[AlishaShorts, AlishaSolidWomensCyclingShorts,...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[s, s, s, additional, alisha, alisha, altht_3p...","[3shorts, fabriccottonlycra, idealforwomens, p...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,[FabHomeDecorFabricDoubleSofaBed(FinishColo......,SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...","[s, s, ..., ..., 22, 646, 6-8, allowed, also, ...","[1sofabed, bedsizedouble, brandfabhomedecor, d...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, ..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[AWBellies, Ballerinas, Footwear, WomensFootwear]",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...","[air, allow, aw, aw, aw, bags, basis, bellies,...","[colorred, heelheight1inch, idealforwomen, occ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, ..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[AlishaShorts, AlishaSolidWomensCyclingShorts,...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[s, s, s, additional, alisha, alisha, altght_1...","[2shorts, fabriccottonlycra, idealforwomens, p...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[Grooming, PetSupplies, Shampoo, SiconsAllPurp...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...","[arnica, arnica, arnica, box, brand, dog, dog,...","[brandsicons, formfactorliquid, fragrancearnic...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [58]:
new_id = list(range(0,flipkart.shape[0]))
flipkart['new_id'] = new_id

In [92]:
def similarity(id1,id2):
    a = flipkart.iloc[id1]
    b = flipkart.iloc[id2]
    
    category_a = a['product_binary']
    category_b = b['product_binary']
    
    score1 = spatial.distance.cosine(category_a,category_b)
    
    description_a = a['description_binary']
    description_b = b['description_binary']
    
    score2 = spatial.distance.cosine(description_a,description_b)
    
    specs_a = a['specs_binary']
    specs_b = b['specs_binary']
    
    score3 = spatial.distance.cosine(specs_a,specs_b)
    
    return score1 + score2 + score3

In [60]:
from scipy import spatial
from operator import itemgetter

In [61]:
flipkart.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,...,product_rating,overall_rating,brand,product_specifications,cleaned_description,specs,product_binary,specs_binary,description_binary,new_id
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[AlishaShorts, AlishaSolidWomensCyclingShorts,...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[s, s, s, additional, alisha, alisha, altht_3p...","[3shorts, fabriccottonlycra, idealforwomens, p...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...",0
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,[FabHomeDecorFabricDoubleSofaBed(FinishColo......,SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...","[s, s, ..., ..., 22, 646, 6-8, allowed, also, ...","[1sofabed, bedsizedouble, brandfabhomedecor, d...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, ...",1
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[AWBellies, Ballerinas, Footwear, WomensFootwear]",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...","[air, allow, aw, aw, aw, bags, basis, bellies,...","[colorred, heelheight1inch, idealforwomen, occ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, ...",2
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[AlishaShorts, AlishaSolidWomensCyclingShorts,...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[s, s, s, additional, alisha, alisha, altght_1...","[2shorts, fabriccottonlycra, idealforwomens, p...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...",3
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[Grooming, PetSupplies, Shampoo, SiconsAllPurp...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...","[arnica, arnica, arnica, box, brand, dog, dog,...","[brandsicons, formfactorliquid, fragrancearnic...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...",4


In [99]:
def product(item_name):
    prod = flipkart[flipkart['product_name'].str.contains(item_name)].iloc[0].to_frame().T    
    print('Selected item :',prod.iloc[0][3])
    def get_neighbors(base,k):
        distances = []
    
        for index, product in flipkart.iterrows():
            if product['new_id'] != base['new_id'].values[0]:
                dist = similarity(base['new_id'].values[0], product['new_id'])
                distances.append((product['new_id'], dist))

        distances.sort(key=itemgetter(1))
        neighbors = []

        for x in range(k):
            neighbors.append(distances[x])
        return neighbors
    
    k = 10
    neighborss = get_neighbors(prod,k)
    print()
    print('Recommended Items :')
    print()
    for neighbor in neighborss:
        print (flipkart.iloc[neighbor[0]][3],"| Retail Price",flipkart.iloc[neighbor[0]][6],"| Discounted Price",flipkart.iloc[neighbor[0]][7])

Remember the function product looks for the first product it finds in the dataset containing the string fed to the function !

In [100]:
product('Sofa Bed')

Selected item : FabHomeDecor Fabric Double Sofa Bed

Recommended Items :

FabHomeDecor Fabric Double Sofa Bed | Retail Price 32157.0 | Discounted Price 22646.0
FabHomeDecor Fabric Double Sofa Bed | Retail Price 32157.0 | Discounted Price 22646.0
FabHomeDecor Fabric Double Sofa Bed | Retail Price 32157.0 | Discounted Price 22646.0
Comfort Couch Engineered Wood 3 Seater Sofa | Retail Price 28625.0 | Discounted Price 19237.0
Stellar Engineered Wood Entertainment Unit | Retail Price 28200.0 | Discounted Price 25380.0
Ethnic Handicrafts Solid Wood Queen Bed | Retail Price 39999.0 | Discounted Price 24999.0
Ethnic Handicrafts Solid Wood Single Bed | Retail Price 36999.0 | Discounted Price 24999.0
Ethnic Handicrafts Solid Wood Queen Bed | Retail Price 39999.0 | Discounted Price 24999.0
RoyalOak Metal Outdoor Chair | Retail Price 10000.0 | Discounted Price 7000.0
IBS Plastic Portable Laptop Table | Retail Price 2499.0 | Discounted Price 1399.0
